<a href="https://colab.research.google.com/github/lstene/charlesLecrerc-performance/blob/main/F1_CL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Charles Lecrec Performance analysis 2018-2023

In [ ]:
! pip install pyspark
! pip install findspark

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

import findspark
findspark.init()

sc = SparkContext()
# Creating a spark session
spark = SparkSession \
  .builder \
  .appName( "Charles Lecrec Performance analysis 2018-2023") \
  .config("spark.some.config.option", " some-value") \
  .getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=9db354a400ebe12114d34562bb8093480112dc938b8e9311b62a34620ed2e922
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from functools import reduce
import pandas as pd
from pyspark.sql.functions import col, sum, to_date, when, udf, mean, round, year
from pyspark.sql.types import StringType
from datetime import datetime
from pyspark.sql import functions as F

First we upload the data from the season 2018 to 2023 in order to analyze how Charles Lecrerc performs during the seasons and unify it into a single pyspark DataFrame

In [ ]:
df_2018=spark.read.csv("season_2018.csv", header=True, inferSchema=True)
df_2019=spark.read.csv("season_2019.csv", header=True, inferSchema=True)
df_2020=spark.read.csv("season_2020.csv", header=True, inferSchema=True)
df_2021=spark.read.csv("season_2021.csv", header=True, inferSchema=True)
df_2022=spark.read.csv("season_2022.csv", header=True, inferSchema=True)
df_2023=spark.read.csv("seasons_2023.csv", header=True, inferSchema=True)

Before merging the dataframes we need to verify they have the same number of columns

In [ ]:
print(len(df_2018.columns))
print(len(df_2019.columns))
print(len(df_2020.columns))
print(len(df_2021.columns))
print(len(df_2022.columns))
print(len(df_2023.columns))

27
27
27
27
27
27


In [ ]:
df_cl = [df_2018, df_2019, df_2020, df_2021, df_2022, df_2023]
df_cl = reduce(lambda df1, df2: df1.union(df2), df_cl)

df_cl.show(5)
print("Number of rows: ",df_cl.count())

+------------+-------------+------------+--------+--------+---------+------+---------+--------+---------------+-----------+-----------+--------+------------------+------------+----+----+----+--------------------+--------+------+-----------+--------------------+---------+-----------+--------------------+----------+
|DriverNumber|BroadcastName|Abbreviation|DriverId|TeamName|TeamColor|TeamId|FirstName|LastName|       FullName|HeadshotUrl|CountryCode|Position|ClassifiedPosition|GridPosition|  Q1|  Q2|  Q3|                Time|  Status|Points|RoundNumber|            RaceName|EventDate|NumPitStops|        TireStrategy|FastestLap|
+------------+-------------+------------+--------+--------+---------+------+---------+--------+---------------+-----------+-----------+--------+------------------+------------+----+----+----+--------------------+--------+------+-----------+--------------------+---------+-----------+--------------------+----------+
|          16|    C LECLERC|         LEC| leclerc|  

## Data exploration

In [ ]:
print(df_cl.printSchema())

root
 |-- DriverNumber: integer (nullable = true)
 |-- BroadcastName: string (nullable = true)
 |-- Abbreviation: string (nullable = true)
 |-- DriverId: string (nullable = true)
 |-- TeamName: string (nullable = true)
 |-- TeamColor: string (nullable = true)
 |-- TeamId: string (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- FullName: string (nullable = true)
 |-- HeadshotUrl: string (nullable = true)
 |-- CountryCode: string (nullable = true)
 |-- Position: double (nullable = true)
 |-- ClassifiedPosition: string (nullable = true)
 |-- GridPosition: double (nullable = true)
 |-- Q1: string (nullable = true)
 |-- Q2: string (nullable = true)
 |-- Q3: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Points: double (nullable = true)
 |-- RoundNumber: integer (nullable = true)
 |-- RaceName: string (nullable = true)
 |-- EventDate: string (nullable = true)
 |-- NumPitStops: inte

In [ ]:
null_values = df_cl.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df_cl.columns)).show()

+------------+-------------+------------+--------+--------+---------+------+---------+--------+--------+-----------+-----------+--------+------------------+------------+---+---+---+----+------+------+-----------+--------+---------+-----------+------------+----------+
|DriverNumber|BroadcastName|Abbreviation|DriverId|TeamName|TeamColor|TeamId|FirstName|LastName|FullName|HeadshotUrl|CountryCode|Position|ClassifiedPosition|GridPosition| Q1| Q2| Q3|Time|Status|Points|RoundNumber|RaceName|EventDate|NumPitStops|TireStrategy|FastestLap|
+------------+-------------+------------+--------+--------+---------+------+---------+--------+--------+-----------+-----------+--------+------------------+------------+---+---+---+----+------+------+-----------+--------+---------+-----------+------------+----------+
|           0|            0|           0|       8|       0|        0|     8|        0|       0|       0|         27|         80|       1|                 1|           0|124|124|124|  45|     8|   

As it can be seen most of the data types must be transformed first in order to continue with any exploratoy analysis. First we are going to eliminate all columns we are not going to use

In [ ]:
columns_to_drop = ["DriverNumber", "BroadcastName", "Abbreviation", "DriverId", "TeamColor", "FirstName", "TeamId"
                   "LastName", "FullName", "HeadshotUrl", "CountryCode", "Q1", "Q2","Q3",
                   "Time", "Status", "RoundNumber"]

df2_cl = df_cl.drop(*columns_to_drop)

df2_cl.show()

+--------+------+--------+--------+------------------+------------+------+--------------------+----------+-----------+--------------------+----------+
|TeamName|TeamId|LastName|Position|ClassifiedPosition|GridPosition|Points|            RaceName| EventDate|NumPitStops|        TireStrategy|FastestLap|
+--------+------+--------+--------+------------------+------------+------+--------------------+----------+-----------+--------------------+----------+
|  Sauber|sauber| Leclerc|    13.0|                13|        18.0|   0.0|Australian Grand ...| 3/25/2018|          2|SUPERSOFT > SOFT ...|     false|
|  Sauber|sauber| Leclerc|    12.0|                12|        19.0|   0.0|  Bahrain Grand Prix|  4/8/2018|          2|SOFT > MEDIUM > S...|     false|
|  Sauber|sauber| Leclerc|    19.0|                19|        19.0|   0.0|  Chinese Grand Prix| 4/15/2018|          1|       SOFT > MEDIUM|     false|
|  Sauber|sauber| Leclerc|     6.0|                 6|        13.0|   8.0|Azerbaijan Grand ...

Now we have to cast the different data types to their correct form.

Firts let's cast the "EvenDate" into date type

In [ ]:
def parse_date(date_str):
    if date_str is None:
        return None
    # Ensure the input is a string and remove leading/trailing spaces
    date_str = str(date_str).strip()
    formats = [
        '%m/%d/%Y',              # Format for '11/3/2019'
        '%Y-%m-%d'               # Format for '2019-11-03'
    ]
    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt).strftime('%Y-%m-%d')
        except ValueError:
            continue
    return None  # Return None if no format matched

In [ ]:
# Register the UDF
parse_date_udf = udf(parse_date, StringType())

# Apply the UDF to convert date formats
df2_cl = df2_cl.withColumn('EventDate', parse_date_udf(col('EventDate')))

In [ ]:
df2_cl = df2_cl.withColumn("EventDate", df2_cl["EventDate"].cast("date"))

Now let's cast the integer columns. It{s important to understand that in the cases where Charles was forced to retire from the race then it is registered with a letter 'R'. When data was uploaded it turn into null so there are a few thing we have to do

In [ ]:
df2_cl.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- TeamId: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Position: double (nullable = true)
 |-- ClassifiedPosition: string (nullable = true)
 |-- GridPosition: double (nullable = true)
 |-- Points: double (nullable = true)
 |-- RaceName: string (nullable = true)
 |-- EventDate: date (nullable = true)
 |-- NumPitStops: integer (nullable = true)
 |-- TireStrategy: string (nullable = true)
 |-- FastestLap: boolean (nullable = true)



Now let's cast the last data type of "ClassifiesPosition" from string to double

In [ ]:
df2_cl = df2_cl.withColumn("ClassifiedPosition", df2_cl["ClassifiedPosition"].cast("double"))

print(df2_cl.printSchema())

root
 |-- TeamName: string (nullable = true)
 |-- TeamId: string (nullable = true)
 |-- LastName: string (nullable = true)
 |-- Position: double (nullable = true)
 |-- ClassifiedPosition: double (nullable = true)
 |-- GridPosition: double (nullable = true)
 |-- Points: double (nullable = true)
 |-- RaceName: string (nullable = true)
 |-- EventDate: date (nullable = true)
 |-- NumPitStops: integer (nullable = true)
 |-- TireStrategy: string (nullable = true)
 |-- FastestLap: boolean (nullable = true)

None


We also have one missing data in "Points". Analyzing the data it's possible to identify that the null value comes from a race were Charles had a DNF so we are going to replace it with a 0

In [ ]:
df2_cl = df2_cl.withColumn("Points", when(col("Points").isNull(), 0.0).otherwise(col("Points")))

null_values = df2_cl.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df2_cl.columns)).show()

+--------+------+--------+--------+------------------+------------+------+--------+---------+-----------+------------+----------+
|TeamName|TeamId|LastName|Position|ClassifiedPosition|GridPosition|Points|RaceName|EventDate|NumPitStops|TireStrategy|FastestLap|
+--------+------+--------+--------+------------------+------------+------+--------+---------+-----------+------------+----------+
|       0|     8|       0|       1|                23|           0|     0|       0|        0|          0|           0|         0|
+--------+------+--------+--------+------------------+------------+------+--------+---------+-----------+------------+----------+



## Create weather variable

As in the FastF1 api there is no information about the type (dry, wet, mixed) we will create this data. For that we will use the tyre strategy to determine the type of data we need.

In [ ]:
df3_cl = df2_cl.withColumn(
    "weather",
    F.when(
        F.col("TireStrategy").rlike("WET|INTERMEDIATE") &
        ~F.col("TireStrategy").rlike("SOFT|HARD|MEDIUM|SUPERSOFT|ULTRASOFT "), "wet"  # Only wet o intermediate
    ).when(
        F.col("TireStrategy").rlike("WET|INTERMEDIATE") &
        F.col("TireStrategy").rlike("SOFT|HARD|MEDIUM|SUPERSOFT|ULTRASOFT "), "mixed"  # Mixed
    ).otherwise("dry")  # None wet or intermediate
)

df3_cl.show()

+--------+------+--------+--------+------------------+------------+------+--------------------+----------+-----------+--------------------+----------+-------+
|TeamName|TeamId|LastName|Position|ClassifiedPosition|GridPosition|Points|            RaceName| EventDate|NumPitStops|        TireStrategy|FastestLap|weather|
+--------+------+--------+--------+------------------+------------+------+--------------------+----------+-----------+--------------------+----------+-------+
|  Sauber|sauber| Leclerc|    13.0|              13.0|        18.0|   0.0|Australian Grand ...|2018-03-25|          2|SUPERSOFT > SOFT ...|     false|    dry|
|  Sauber|sauber| Leclerc|    12.0|              12.0|        19.0|   0.0|  Bahrain Grand Prix|2018-04-08|          2|SOFT > MEDIUM > S...|     false|    dry|
|  Sauber|sauber| Leclerc|    19.0|              19.0|        19.0|   0.0|  Chinese Grand Prix|2018-04-15|          1|       SOFT > MEDIUM|     false|    dry|
|  Sauber|sauber| Leclerc|     6.0|           

### Create type of circuit variable

Also, for a better understanding of Charles' performance it's necesary to view his performance over the different type of circuits that F1 offers. For that we are going to create a new variable called "typeCircuit" that will have values of


*   Permanent circuits: These are tracks built exclusively for motor racing
*   Urban circuits: They are run on city streets and usually have closer barriers.
* Hybrid circuits: These are a combination of street streets and tracks built for the race.

For that let's see all te values that existe in te colunm RaceName



In [ ]:
df3_cl.select("RaceName").distinct().show(truncate=False,n=50)

+---------------------------+
|RaceName                   |
+---------------------------+
|German Grand Prix          |
|Spanish Grand Prix         |
|Brazilian Grand Prix       |
|Mexican Grand Prix         |
|Singapore Grand Prix       |
|Canadian Grand Prix        |
|Italian Grand Prix         |
|Chinese Grand Prix         |
|Azerbaijan Grand Prix      |
|British Grand Prix         |
|Monaco Grand Prix          |
|Russian Grand Prix         |
|Austrian Grand Prix        |
|Australian Grand Prix      |
|Hungarian Grand Prix       |
|Bahrain Grand Prix         |
|French Grand Prix          |
|Abu Dhabi Grand Prix       |
|Belgian Grand Prix         |
|United States Grand Prix   |
|Japanese Grand Prix        |
|Sakhir Grand Prix          |
|Turkish Grand Prix         |
|Portuguese Grand Prix      |
|Emilia Romagna Grand Prix  |
|Tuscan Grand Prix          |
|Eifel Grand Prix           |
|Styrian Grand Prix         |
|70th Anniversary Grand Prix|
|São Paulo Grand Prix       |
|Dutch Gra

In [ ]:
category_dict = {
    "permanent": [
        "German Grand Prix", "Spanish Grand Prix", "Brazilian Grand Prix",
        "Mexican Grand Prix", "Canadian Grand Prix", "Italian Grand Prix",
        "Chinese Grand Prix", "Russian Grand Prix", "Austrian Grand Prix",
        "Hungarian Grand Prix", "Bahrain Grand Prix", "French Grand Prix",
        "Abu Dhabi Grand Prix", "Belgian Grand Prix", "United States Grand Prix",
        "Japanese Grand Prix", "Sakhir Grand Prix", "Turkish Grand Prix",
        "Portuguese Grand Prix", "Emilia Romagna Grand Prix", "Tuscan Grand Prix",
        "Eifel Grand Prix", "Styrian Grand Prix", "70th Anniversary Grand Prix",
        "São Paulo Grand Prix", "Dutch Grand Prix", "Qatar Grand Prix",
        "Mexico City Grand Prix","British Grand Prix"
    ],
    "urban": [
        "Monaco Grand Prix", "Azerbaijan Grand Prix", "Singapore Grand Prix",
        "Las Vegas Grand Prix", "Saudi Arabian Grand Prix", "Miami Grand Prix"
    ],
    "hybrid": [
        "Australian Grand Prix", "Canadian Grand Prix", "Russian Grand Prix",
        "Abu Dhabi Grand Prix", "70th Anniversary Grand Prix"
    ],
}


In [ ]:
def categorize_circuits(df, column_name, category_dict):

    df = df.withColumn("circuit_category", F.lit(None))
    for category, circuits in category_dict.items():
        regex = "|".join(circuits)
        df = df.withColumn(
            "circuit_category",
            F.when(F.col(column_name).rlike(regex), category).otherwise(F.col("circuit_category"))
        )

    return df


In [ ]:
df3_cl = categorize_circuits(df3_cl, "RaceName", category_dict)
df3_cl.show()

+--------+------+--------+--------+------------------+------------+------+--------------------+----------+-----------+--------------------+----------+-------+----------------+
|TeamName|TeamId|LastName|Position|ClassifiedPosition|GridPosition|Points|            RaceName| EventDate|NumPitStops|        TireStrategy|FastestLap|weather|circuit_category|
+--------+------+--------+--------+------------------+------------+------+--------------------+----------+-----------+--------------------+----------+-------+----------------+
|  Sauber|sauber| Leclerc|    13.0|              13.0|        18.0|   0.0|Australian Grand ...|2018-03-25|          2|SUPERSOFT > SOFT ...|     false|    dry|          hybrid|
|  Sauber|sauber| Leclerc|    12.0|              12.0|        19.0|   0.0|  Bahrain Grand Prix|2018-04-08|          2|SOFT > MEDIUM > S...|     false|    dry|       permanent|
|  Sauber|sauber| Leclerc|    19.0|              19.0|        19.0|   0.0|  Chinese Grand Prix|2018-04-15|          1|  